In [ ]:
# Load all libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from math import sqrt
import itertools

# Load train dataset
df = pd.read_csv('train.csv')
# Load features dataset and join it with train data
features_df = pd.read_csv('features.csv')
df = pd.merge(df, features_df.drop(['IsHoliday'], axis = 1), how = 'left', on = ['Store', 'Date'])
# Load store dataset and join with above data
stores_df = pd.read_csv('stores.csv')
df = pd.merge(df, stores_df, how = 'left', on = ['Store'])

# Filter data for store 20
df = df[df['Store'] == 20]

# Impute NULL values
df['MarkDown1'] = df['MarkDown1'].fillna(0)
df['MarkDown2'] = df['MarkDown2'].fillna(0)
df['MarkDown3'] = df['MarkDown3'].fillna(0)
df['MarkDown4'] = df['MarkDown4'].fillna(0)
df['MarkDown5'] = df['MarkDown5'].fillna(0)

# Create year, month, and date
df['Date'] = pd.to_datetime(df['Date'])
df['month_date'] = df['Date'].apply(lambda i : i.month)
df['day_date'] = df['Date'].apply(lambda i : i.day)
df['year_date'] = df['Date'].apply(lambda i : i.year)

# One hot encoding
cols_to_encode = ['Type', 'IsHoliday']
df = pd.get_dummies(data = df, columns = cols_to_encode, drop_first = True)

# Standard Scaler
standard_scaler = StandardScaler()
feature_cols = ['Temperature', 'Fuel_Price', 'MarkDown1','MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'Size']
transformed_features = standard_scaler.fit_transform(df[feature_cols])
df[feature_cols] = transformed_features

# Initialize final forecast series
final_forecast = pd.Series(dtype='float64')
final_actual = pd.Series(dtype='float64')

# Initialize performance metrics
performance_metrics = []

# Iterate through each department in store 20
departments = df['Dept'].unique()
for dept in departments:
    dept_df = df[df['Dept'] == dept]
    
    # Time Series Analysis
    sales_data = dept_df.groupby('Date')['Weekly_Sales'].sum().sort_index()
    
    # Perform the Augmented Dickey-Fuller test to check for stationarity
    adf_test = adfuller(sales_data)
    adf_p_value = adf_test[1]
    
    # Determine the number of lags
    max_lags = min(40, len(sales_data) // 2)
    
    # Define train and test sets
    train_size = int(len(sales_data) * 0.8)
    train = sales_data[:train_size]
    test = sales_data[train_size:]
    
    # Auto ARIMA to find best p, d, q
    p = d = q = range(0, 3)
    pdq = list(itertools.product(p, d, q))
    
    # Evaluate models
    best_aic = float("inf")
    best_pdq = None
    best_model = None
    
    for param in pdq:
        try:
            temp_model = ARIMA(train, order=param)
            temp_result = temp_model.fit()
            if temp_result.aic < best_aic:
                best_aic = temp_result.aic
                best_pdq = param
                best_model = temp_result
        except:
            continue
    
    # Forecast using the best model
    forecast_steps = len(test)
    forecast = best_model.get_forecast(steps=forecast_steps)
    forecast_df = forecast.summary_frame()
    forecast_values = forecast_df['mean']
    
    # Aggregate the forecast and actual values
    if final_forecast.empty:
        final_forecast = forecast_values
        final_actual = test
    else:
        final_forecast = final_forecast.add(forecast_values, fill_value=0)
        final_actual = final_actual.add(test, fill_value=0)
    
    # Calculate performance metrics for the best model
    mse = mean_squared_error(test, forecast_values)
    rmse = sqrt(mse)
    mae = mean_absolute_error(test, forecast_values)
    mape = np.mean(np.abs((test - forecast_values) / test)) * 100
    r2 = r2_score(test, forecast_values)
    
    performance_metrics.append({
        'Dept': dept,
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'MAPE': mape,
        'R2': r2
    })

# Save performance metrics to a file
performance_df = pd.DataFrame(performance_metrics)
performance_df.to_csv('performance_metrics.csv', index=False)

# Save summary of the best model for the last department processed
with open('best_model_summary.txt', 'w') as f:
    f.write("Best Model Summary:\n")
    f.write(str(best_model.summary()))
    f.write(f"\nBest pdq: {best_pdq}\n")

# Calculate performance metrics for the final aggregated model
final_mse = mean_squared_error(final_actual, final_forecast)
final_rmse = sqrt(final_mse)
final_mae = mean_absolute_error(final_actual, final_forecast)
final_mape = np.mean(np.abs((final_actual - final_forecast) / final_actual)) * 100
final_r2 = r2_score(final_actual, final_forecast)

# Save final aggregated performance metrics to a file
with open('final_aggregated_performance_metrics.txt', 'w') as f:
    f.write(f"Final Aggregated MSE: {final_mse}\n")
    f.write(f"Final Aggregated RMSE: {final_rmse}\n")
    f.write(f"Final Aggregated MAE: {final_mae}\n")
    f.write(f"Final Aggregated MAPE: {final_mape}\n")
    f.write(f"Final Aggregated R2: {final_r2}\n")

# Plot the final aggregated forecast
plt.figure(figsize=(12, 6))
plt.plot(final_forecast.index, final_forecast, label='Aggregated Forecast', color='blue')
plt.plot(final_actual.index, final_actual, label='Actual Sales', color='orange')
plt.title('Aggregated ARIMA Model Forecast for Store 20')
plt.xlabel('Date')
plt.ylabel('Weekly Sales')
plt.legend()
plt.savefig('aggregated_forecast_plot.png')
plt.close()